In [14]:
a = """
.subckt psquid1        3  5 
L1                 3         1   #@SQUIDL1(1.75)pH fcheck
L2                 3         4   #@SQUIDL1(v=0.65,fix=true,lic=L2,bc=l2,global=32)pH fcheck
B1                 1         5  jjmod area=#JJAREA1( v=0.49, fix = true,lic=L1,bc=P2,global=43)
R1                 1         5   #@JJSHUNT1(16)ohm
B2                 2         5  jjmod area=#JJAREA1(0.49)
R2                 2         5   #@JJSHUNT1(16)ohm
B3                 4         2  pjjmod area=#PJJAREA1(20)
"""

In [24]:
def isint(s):  # 正規表現を使って判定を行う
    p = '[-+]?\d+'
    return True if re.fullmatch(p, s) else False

def isfloat(s):  # 正規表現を使って判定を行う
    p = '[-+]?(\d+\.?\d*|\.\d+)([eE][-+]?\d+)?'
    return True if re.fullmatch(p, s) else False

def digit(s):  # 正規表現を使って小数点以下の桁数
    if re.search("\.",s)!= None:
        return len(re.split("\.",s)[1])
    else:
        return 0

def stringToNum(s):
    if isint(s):
        return int(s)

    elif isfloat(s):
        return float(s)

    else:
        raise ValueError("値が数値ではありません。")

In [20]:
import re
import pandas as pd

df = pd.DataFrame(columns=['char','text','default','value','fix','lic','bc','global'])  
df.set_index('char', inplace=True)
b = re.findall('#.+\(.+\)',a)
b = re.sub('\s','',b)
  


In [21]:
print(df)

Empty DataFrame
Columns: [text, default, value, fix, lic, bc, global]
Index: []


In [36]:
for l in b:
    dic = {'char': None,'text': None,'default': None,'value': None,'fix': False,'lic': None,'bc': None,'global': None}
    char = re.search('#.+\(',l, flags=re.IGNORECASE).group()
    char = re.sub('#|\(','',char)
    dic['char']=char
    dic['text']='#'+char
    m = re.search('\(.+\)',l).group()
    m = re.sub('\(|\)','',m)
    spl = re.split(',',m)
    if len(spl)==1:
        if isfloat(spl[0]) or isint(spl[0]):
            num = stringToNum(spl[0])
            dic['default'] = num
            dic['value'] = num
    for sp in spl:
        val = re.split('=',sp)
        if len(val) == 1:
            if isfloat(val[0]) or isint(val[0]):
                num = stringToNum(spl[0])
                dic['default'] = num
                dic['value'] = num
        elif len(val) != 2:
            print(len(val))
            raise ValueError("[ "+sp+" ]の記述が読み取れません。",len(val))

        if re.fullmatch('v|value',val[0],flags=re.IGNORECASE):
            num = stringToNum(val[1])
            dic['default'] = num
            dic['value'] = num
        elif re.fullmatch('fix|fixed',val[0],flags=re.IGNORECASE):
            if re.fullmatch('true',val[1],flags=re.IGNORECASE):
                dic['fix'] = True
            else:
                dic['fix'] = False
        elif re.fullmatch('lic',val[0],flags=re.IGNORECASE):
            if re.fullmatch('(L|Ic|I)\d+',val[1],flags=re.IGNORECASE):
                dic['lic'] = val[1]
            else:
                raise ValueError("[ "+sp+" ]のLIc積の記述が読み取れません。")
        elif re.fullmatch('(β|b)c',val[0],flags=re.IGNORECASE):
            if re.fullmatch('(A|R)\d+',val[1],flags=re.IGNORECASE):
                dic['bc'] = val[1]
            else:
                raise ValueError("[ "+sp+" ]のbcの記述が読み取れません。")
        elif re.fullmatch('global',val[0],flags=re.IGNORECASE):
            num = stringToNum(val[1])
            dic['global'] = num
        else:
            raise ValueError("[ "+sp+" ]の記述が読み取れません。")
    print(dic)

ValueError: [ 1.75 ]の記述が読み取れません。